In [14]:
import pandas as pd
import numpy as np
import scipy
import re
import spacy
import matplotlib.pyplot as plt 
import math
from dotenv import load_dotenv
import statsmodels.api as sm
import seaborn as sns
import textwrap
from tabulate import tabulate
from prettytable import PrettyTable
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess
from gensim import corpora
from pprint import pprint
from gensim.test.utils import common_texts
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import tqdm
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lucyjhampton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import os
os.chdir(r'/Users/lucyjhampton/Library/Mobile Documents/com~apple~CloudDocs/Jupyter/AI Bib')

In [16]:
## GET PAPERS

papers = pd.read_csv("no_duplicates.csv")
papers

,Authors or Inventors,Publication Year,Article Title,DOI,Source Title,Abstract
0,Lu C.; Yu F.; Li J.; Li S.,2024,Research on safe-haven currencies under global...,10.1016/j.gfj.2024.101013,Global Finance Journal,The backdrop of this research is the high glob...
1,Farhoudinia B.; Ozturkcan S.; Kasap N.,2024,Emotions unveiled: detecting COVID-19 fake new...,10.1057/s41599-024-03083-5,Humanities and Social Sciences Communications,The COVID-19 pandemic has highlighted the pern...
2,Vaz E.,2024,Quantum machine learning in spatial analysis: ...,10.1007/s12076-024-00374-y,Letters in Spatial and Resource Sciences,This paper explores the ground-breaking approa...
3,Arthur L.; Vondolia G.K.; Dasmani I.,2024,Superstition and attitudes towards restoration...,10.1016/j.forpol.2024.103297,Forest Policy and Economics,Forest restoration has become a worldwide phen...
4,Huang Y.,2024,A theory of emotion based on a universal model,10.1057/s41599-024-02869-x,Humanities and Social Sciences Communications,The complexity of emotions has thus far limite...
...,...,...,...,...,...,...
39755,"Iurasov, A; Stanelyte, G",2020,STUDY OF DIFFERENT DATA SCIENCE METHODS FOR DE...,10.3846/bm.2020.604,11TH INTERNATIONAL SCIENTIFIC CONFERENCE BUSIN...,The demand prediction becoming an essential to...
39756,"Monat, JP",2024,The self-awareness of the forest,10.1016/j.futures.2024.103429,FUTURES,Systems Thinking theorist J. P. Monat has hypo...
39757,"Zhou, Y; Fan, JQ; Xue, LR",2024,How Much Can Machines Learn Finance from Chine...,10.1287/mnsc.2022.01468,MANAGEMENT SCIENCE,How much can we learn finance directly from te...
39758,"Lin, WB",2010,Relevant factors that affect service recovery ...,10.1080/02642060801935739,SERVICE INDUSTRIES JOURNAL,This paper explores the factors that affect em...


In [52]:
example1 = """We investigate the potential implications of large language models (LLMs), such as Generative Pre-trained Transformers (GPTs), on the U.S. labor market, focusing on the increased capabilities arising from LLM-powered software compared to LLMs on their own. Using a new rubric, we assess occupations based on their alignment with LLM capabilities, integrating both human expertise and GPT-4 classifications. Our findings reveal that around 80% of the U.S. workforce could have at least 10% of their work tasks affected by the introduction of LLMs, while approximately 19% of workers may see at least 50% of their tasks impacted. We do not make predictions about the development or adoption timeline of such LLMs. The projected effects span all wage levels, with higher-income jobs potentially facing greater exposure to LLM capabilities and LLM-powered software. Significantly, these impacts are not restricted to industries with higher recent productivity growth. Our analysis suggests that, with access to an LLM, about 15% of all worker tasks in the US could be completed significantly faster at the same level of quality. When incorporating software and tooling built on top of LLMs, this share increases to between 47 and 56% of all tasks. This finding implies that LLM-powered software will have a substantial effect on scaling the economic impacts of the underlying models. We conclude that LLMs such as GPTs exhibit traits of general-purpose technologies, indicating that they could have considerable economic, social, and policy implications."""

output1 = """The paper focuses on predicting the labour market impacts of AI, an economic outcome. It uses GPT-4 classifications to estimate occupational AI exposure. Class: A"""

example2 = """The use of Markov processes (or Markov chains) has become widespread in dynamic stochastic modeling. For example, its use is ubiquitous in macroeconomics (dynamic stochastic general equilibrium), finance (dynamic asset pricing), and areas of microeconomics (dynamic programming). As we discuss below, its application in dynamic land use has been more limited, but is, in principle, no less applicable. Using a multi-nominal logit (ML) specification together with serial data on agricultural land use from California, we estimate Markov transition probabilities conditional on number of exogenous factors. Applying so-called “first step” analysis, these transition probabilities are used to forecast the distribution of agricultural crops, which in turn can be used for policy making."""

output2 = """This paper focuses on developing a dynamic stochastic modelling method. It applies this to modelling land use. While land use may have productivity implications, the paper is not focused on analysing the impact of AI on this or any other relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B."""

example3 = """We model Artificial Intelligence (AI) as self-learning capital: Its productivity rises by its use and by training with data. In a three-sector model, an AI sector and an applied research (AR) sector produce intermediates for a final good firm and compete for high-skilled workers. AR development benefits from inter-temporal spillovers and knowledge spillovers of agents working in AI, and AI benefits from application gains through its use in AR. … We show that suitable tax policies induce socially optimal movements of workers between sectors. In particular, we provide a macroeconomic rationale for an AI-tax on AI-producing firms, once the accumulation of AI has sufficiently progressed."""

output3 = """This paper develops a theoretical model of AI as capital and discusses tax policy implications. This model is relevant to the labour market and growth, and discusses economic policy implications, either of which qualify it to be in A. Class: A."""

example4 = """The COVID-19 crisis has had a tremendous economic impact for all countries. Yet, assessing the full impact of the crisis has been frequently hampered by the delayed publication of official GDP statistics in several emerging market and developing economies. This paper outlines a machine-learning framework that helps track economic activity in real time for these economies. As illustrative examples, the framework is applied to selected sub-Saharan African economies. The framework is able to provide timely information on economic activity more swiftly than official statistics."""

output4 = """This paper proposes a method to predict the economic impact of Covid-19. It uses a machine-learning framework to generate real-time forecasts. It is not focused on analysing the impact of AI on any economically relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B"""

example5 = """Innovations in statistical technology, including in predicting creditworthiness, have sparked concerns about distributional impacts across categories such as race. Theoretically, distributional consequences of better statistical technology can come from greater flexibility to uncover structural relationships, or from triangulation of otherwise excluded characteristics. Using data on US mortgages, we predict default using traditional and machine learning models. We find that Black and Hispanic borrowers are disproportionately less likely to gain from the introduction of machine learning. In a simple equilibrium credit market model, machine learning increases disparity in rates between and within groups; these changes are primarily attributable to greater flexibility."""

output5 = """The paper analyses the distributional impacts of the adoption of AI for the prediction of mortgage defaults. It does so using an ML model. It focuses on economically-relevant ethical implications of AI, which qualifies it to be in class A. Class: A."""

example6 = """This paper introduces a novel geo-connected hierarchical deep learning framework for enhanced flood risk modeling, leveraging high-resolution meteorological and hydraulic data. Our method adopts a geographically structured hierarchical deep learning model, providing a more refined analysis of flood risks. In a case study using the Mississippi River, we demonstrate superior performance over traditional deep learning models in terms of accuracy and predictive power. Significantly, when applied to the National Flood Insurance Program’s policy and claim data, our model not only yields more precise risk pricing but also effectively reduces the solvency capital requirement by aligning net premiums more closely with actual risks. The framework’s global optimization approach effectively harnesses spatial and temporal data, while its physics-based structure offers improved interpretability and addresses the complex challenges of anthropogenic influences in flood risk assessment."""

output6 = """This paper focuses on modelling flood risk. It does so using a deep learning model. It is not focused on analysing the impact of AI on any economically relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B"""

example7 = """Leveraging the blood-making function of public transfer payments and stimulating the endogenous motivation of beneficiaries are key focus for poverty alleviation worldwide. Based on the data from 2014‚2018 China Family Panel Studies (CFPS) and machine learning algorithms, this article comprehensively evaluate the effect of Chinese public transfer payment system on labour supply. We find that public transfer payments oriented households increase the labour supply of family members. Decomposing the comprehensive effect, public transfer payments increase the employment probability at the extensive margin and extend the working hours at the intensive margin. Mechanism analysis indicates that public transfer payments is conducive to enhance health capital, promote labour skill training, and relax constraints on infant and child care, thus improving labour supply. Furthermore, groups in female, young, and education below high school show a higher incentive effect for labour supply."""

output7 = """This paper focuses on making public transfer payments more effective at poverty alleviation. It uses a machine learning technique to evaluate the effect of Chinese public transfer payments on labour supply. While the topic is economically relevant, it is not about the impact of AI - it just uses a machine learning technique. Class: B"""

example8 = """Artificial intelligence (AI) and machine learning (ML) algorithms have transformed various industries, including healthcare. Healthcare organizations are now using AI and ML algorithms to drive strategic leadership and decision-making, as they provide insights that help organizations manage resources, improve patient outcomes, and increase efficiency. This research paper examines how AI and ML algorithms are used in healthcare to drive strategic leadership. The paper also explores the benefits and challenges associated with using these technologies in healthcare. The study found that AI and ML algorithms can help healthcare organizations make data-driven decisions, optimize resource allocation, and improve patient outcomes. However, there are still challenges related to data quality and privacy that must be addressed to ensure that AI and ML algorithms are used effectively in healthcare."""

output8 = """This paper focuses on the adoption of algorithms in a particular sector, healthcare. It discusses the adoption or diffusion of AI and discusses relevant management outcomes (strategic leadership and decision-making). Class: A"""

example9 = """General purpose technologies (GPTs) such as AI enable and require significant complementary investments, including co-invention of new processes, products, business models and human capital. These complementary investments are often intangible and poorly measured in the national accounts, even when they create valuable assets for the firm. We develop a model that shows how this leads to an underestimation of productivity growth in the early years of a new GPT, and how later, when the benefits of intangible investments are harvested, productivity growth will be overestimated. Our model generates a Productivity J-Curve that can explain the productivity slowdowns often accompanying the advent of GPTs, as well as the increase in productivity later. We use our model to analyze empirically the historical roles of intangibles tied to R&D, software, and computer hardware. We find substantial and ongoing Productivity J-Curve effects for software in particular and computer hardware to a lesser extent. Our adjusted measure TFP is 11.3% higher than official measures at the end of 2004, and 15.9% higher than official measures at the end of 2017. We then assess how AI-related intangible capital may be currently affecting measured productivity and find the effects are small but growing."""

output9 = """This paper is about AI and a productivity J-curve. It develops a model to analyse this. It is about the measurement challenges AI poses as well as the impact of AI on productivity. Class: A"""

example10 = """Deep learning, the most important subfield of machine learn- ing and artificial intelligence (AI) over the last decade, is considered one of the fundamental technologies underpinning the Fourth Industrial Revolution. But despite its record-breaking history, deep learning’s enormous appetite for compute and data means that sometimes it can be too costly to practically use. In this paper, we connect technical insights from deep learning scaling laws and transfer learning with the economics of IT to propose a framework for estimating the cost of deep learning computer vision systems that achieve a desired level of accuracy. Our tool can be of practical use to AI practitioners in industry or academia to guide investment decisions."""

output10 = """This paper is about one of the drivers of progress in deep learning (i.e., data and compute). The paper uses scaling laws to estimate the cost of deep learning computer vision systems. This is about the economic drivers of AI progress. Class: A"""

example11 = """The paper explores how Generative AI intersects with Competition Law, focusing on Foundation Models (FMs) and Large Language Models (LLMs). It examines industry dynamics and identifies key competition issues like entry barriers, tying, leveraging, and acquisitions. It highlights the supply chain's importance and looks at how FMs are integrated into search software, chatbots, and productivity tools, particularly noting entry barriers such as computing power and data collection. It suggests that FMs might require new approaches to market delineation, possibly creating a separate relevant market for data. The paper also discusses various cases pertaining to tying and leveraging and highlights the difficulty in proving tying due to the blurred lines between traditional search engines and AI chatbots. It illustrates how competition assessments for acquisitions may require changes due to data being a highly flexible commodity for the industry. The paper concludes by calling for increased scrutiny and regulation for the industry. Â© 2024 Informa UK Limited, trading as Taylor & Francis Group."""

output11 = """This paper is about the key competition law issues arising from Generative AI, including entry barriers, acquisitions, supply chains, and the role of data. It makes policy recommendations. It is about AI's implications for policy that is relevant to its economic impacts (namely on competition). Class: A""" 

example12 = """High-frequency trading is prevalent, where automated decisions must be made quickly to take advantage of price imbalances and patterns in price action that forecast near-future movements. Although many algorithms have been explored and tested, analytical methods fail to harness the whole nature of the market environment by focusing on a limited domain. With the ever-growing machine learning field, many large-scale end-to-end studies on raw data have been successfully employed to increase the domain scope for profitable trading but are very difficult to replicate. Combining deep learning on the order books with reinforcement learning is one way of breaking down large-scale end-to-end learning into more manageable and lightweight components for reproducibility suitable for retail trading. The following article focuses on forecasting returns across multiple horizons using order flow imbalance and training three temporal-difference learning models for five financial instruments to provide trading signals. Through backtesting and forward testing, the results prove potential but require further minimal modifications for consistently profitable trading to fully handle retail trading costs, slippage, and spread fluctuation. Â© 2024 With Intelligence LLC."""

output12 = """This paper focuses on using machine learning, specifically deep learning and reinforcement learning, to improve high-frequency trading strategies. The outcome it explores is forecasted returns across multiple time horizions. The paper focuses on developing a machine learning technique for financial forecasting and does not discuss wider implications for economic outcomes, measurement, AI progress, policy or ethics. Class: B"""

example13 = """The General Data Protection Regulation (GDPR) of the European Union has established regulations on automated decisions in Article 22 with the proliferation of artificial intelligence. In response, the Personal Information Protection Act (PIPA) of South Korea, serving as a counterpart to the GDPR, has recently incorporated provisions for automated decisions under Article 37-2 through an amendment. Although the PIPA follows a distinct legal framework from the GDPR, it is crucial to ensure an equivalent level of protection for fundamental rights. Recognising this concern, this study analyses the differences between the PIPA and GDPR regarding automated decisions, focusing on three aspects: format, target, and content. This analysis identifies that the PIPA lacks comprehensive safeguards for data subjects in certain aspects compared to the GDPR. First, regarding the format, the PIPA grants the right to object rather than establishing a general prohibition to automated decisions, posing limitations in protecting individuals who are unable to effectively exercise their rights. Second, in terms of the target, the PIPA regulates a completely automated status at the overall system level, creating a regulatory vacuum for a multi-stage profiling system. Third, concerning the content, the PIPA faces several technical and practical limitations that remain unresolved in delineating the content of the right to explanation. Building upon this analysis, this study proposes potential legislation and interpretation remedies to address these concerns based on each aspect. Â© The Author(s) 2024."""

output13 = """This paper analyses the differences between the EU General Data Protection Regulation (GDPR) and South Korea's Personal Information Protection Act (PIPA) with regards to automated decision-making from AI. It proposes potential legislation and interpretation remedies to overcome the shortcomings of each regulation. While it discusses the policy implications of AI, the paper does not focus on the links between these policies and economic outcomes. Class: B"""

example14 = """Code reviews are one of the first quality assurance tasks in continuous software integration and delivery. The goal of our work is to reduce the need for manual reviews by automatically identify which code fragments should be further reviewed manually. We conducted an action research study with two companies where we extracted code reviews and build machine learning classifiers (AdaBoost and Convolutional Neural Network- CNN). Our results show that the accuracy of recognizing code fragments that require manual review, measured with Matthews Correlation Coefficient, was 0.70 in the combination of our own feature extraction and CNN. We conclude that this way of combining automation with manual code reviews can improve the speed of reviews while providing organizations with the possibility to support knowledge transfer among the designers."""

output14 = """This paper analyses the automation of code reviews using AI. While code review is not itself an economic outcome, it is a contributor to software developer productivity, and the paper does focus on evaluating the magnitude of these gains. As such it is hard to classify. Class: C""" 

In [19]:
prompt = "You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. \n\nCategory A: The paper studies: \n- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. \n- The measurement of the economic impact of AI, including measurement challenges. \n- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship between AI, racial inequality and income inequality. \n- The adoption of AI by firms, the government, or sectors; what affects adoption decisions; and how AI is used. \n- Economic drivers of AI progress, such as cost of compute. \nProvided they meet the earlier requirements, include both theoretical and empirical, micro and macro studies from the fields of economics, management, finance, and innovation (including sector-specific studies). \n\nCategory B: The paper does not focus on the above topics. Category B includes papers that focus on developing machine learning techniques to predict financial returns. Category B also includes papers that focus on social outcomes such as race, religion and politics where the paper does not make explicit links to economic outcomes, such as economic inequality, unemployment or resource distribution. Also include papers that are not about the impact or implications of AI, e.g., those that focus on developing or using AI econometric techniques other than to analyse the impact of implications of AI. \n\nCategory C: It is difficult to classify the paper between A and B based on the abstract. \n\nInstructions: 1. Reason first, and summarise in no more than three bullet points, the main focus and methodology of the paper outlined in the abstract. 2. Only when you have enough information, decide on a classification based on 1. Do not use more than 100 words in your response. When giving your final classification, format it as 'Class: x' where x is your suggested classification. For each abstract you correctly classify, I will pay you $10 as a tip."


In [7]:
print(prompt)

You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. 

Category A: The paper studies: 
- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. 
- The measurement of the economic impact of AI, including measurement challenges. 
- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship bet

In [10]:
# test_set = ["""Recent advances in AI represent improvements in prediction. We examine how decision-making and risk management strategies change when prediction improves. The adoption of AI may cause substitution away from risk management activities used when rules are applied (rules require always taking the same action), instead allowing for decision-making (choosing actions based on the predicted state). We provide a formal model evaluating the impact of AI and how risk management, stakes, and interrelated tasks affect AI adoption. The broad conclusion is that AI adoption can be stymied by existing processes designed to address uncertainty. In particular, many processes are designed to enable coordinated decision-making among different organizational actors. AI can make coordination even more challenging. However, when the cost of changing such processes falls, then the returns from AI adoption increase.""", """In recent times, the advent of AI-based optical character recognition (OCR) has garnered significant attention in the realm of digital text conversion. However, it is imperative to note that OCR solely identifies individual characters or words, and lacks the ability to reunite them into cohesive units such as words or sentences. Consequently, the manual sorting of them to establish the appropriate reading order has emerged as a bottleneck. In this paper, we present an algorithm termed adjacent character detection (ACD), designed to serve as a post-processing of OCR, facilitating automatic digital text conversion. The algorithm involves line segmentation through a quad-ACD scan (up-down-down-up), allowing it to consecutively discern characters within a column based on their adjacency relations. Conventional projection profile analyses have struggled to effectively partition the distinct internal structure of Chinese historical text, where two annotation columns often subdivide from a single body column. In contrast, our ACD algorithm employs an approach, reuniting adjacent characters rather than fragmenting the entire text into isolated entities. Additionally, ACD algorithm enabled body/annotation classification for OCR-detected characters based on the pattern analysis of its quad scan. This cumulative information empowers the conversion of digital text in a desired reading order. To assess the efficacy of the proposed algorithm, a set of ground-truth OCR result was subjected to rigorous testing, culminating in a reading order accuracy of 98.6%. Noteworthy robustness was also demonstrated in the face of misaligned columns, experimentally induced by applying tilt, warp, and wavy noises to the original digital images. Lastly, the algorithm was integrated with two pre-developed OCR models, resulting in a reading order accuracy of 97.7%.""", """Artificial intelligence (AI) may be the next general purpose technology. General purpose technologies, such as the steam engine and computing, can have an outsized impact on productivity through a positive feedback loop between producing and application industries. Along with the discussion of AI's potential to improve productivity come a number of policy concerns related to AI's potential to automate jobs and to create existential risk for humanity. Because of these worries, in March 2023, a widely circulated petition called for a pause in AI research. That letter asked several questions about AI's potential impact on society. This paper examines those questions through an economic lens. It highlights reasons to be optimistic about the long-run impact of AI, while underscoring short-run risks. Economic models provide an understanding of where the ambiguity lies and where it does not. Our models suggest no ambiguity on whether there will be jobs and little ambiguity on long-term productivity growth if AI diffuses widely. In contrast, there is substantial ambiguity on the implications of AI's diffusion for inequality.""", """This paper critically explores the role of ChatGPT in higher education, with a particular emphasis on preserving the academic integrity of student assessments via a risk management paradigm. A literature analysis was conducted to understand existing strategies for addressing academic misconduct and the necessity of equipping students with skills suitable for an AI-driven workforce. The paper's unique contribution lies in its use of a risk management approach to enable educators to identify potential risks, devise mitigation strategies, and ultimately apply a proposed conceptual framework in educational environments. The paper concludes with identifying practical limitations and proposed future research areas, focusing on the uncertainties that emerge from the evolution of AI LLMs and the integration of comprehensive AI tools that pose new risks and opportunities.""", """The application of Artificial Intelligence is being applied in the medical industry at a quick pace, and it is currently serving as the main source of support for clinical practice solutions. Clinical practice accuracy could be improved and costs could be decreased with the use of deep learning techniques. To diagnose Diabetic Retinopathy, an effective and dependable method for automatic screening must be identified. However, deep-learning models may face difficulties due to a lack of data in several medical fields. The Diabetic Retinopathy Detection Model (DRDM), a deep learning model, is proposed in this research to identify retinal images as either infected or uninfected. The data transformation approach is used to address the lack of Diabetic Retinopathy data, which helps prevent overfitting by doubling the data. The paper shows that building a highly complex model like EfficientNetB3 or VGG16 is not necessary to achieve high performance, where, the experiment's test results approved that the DRDM model outperforms such pre-trained models. Furthermore, it took much less time for the DRDM model to produce these results""", """Artificial Intelligence (AI) technology is changing the industrial paradigm and has been widely adopted in port operations. Although AI technology can improve the efficiency of port operations and service quality, ports need to bear some costs. Discussing the role of applying AI technology to ports in complex competitive environments has become an important issue in the operations of ports and shipping. In this study, we construct a game-theoretic model of competitive heterogeneous ports. The research conclusions indicate that the adoption of AI technology by heterogeneous ports can enhance port profits. Unfortunately, simultaneous adoption exacerbates homogenized competition, posing a threat to profit realization. Furthermore, while the hub port can leverage AI-empowered capabilities to strengthen own competitiveness, it can undermine the performance of competitors and society at large. Surprisingly, the adoption of AI technology by feeder port is more advantageous in achieving social welfare and achieving multiple benefits such as carbon reduction.""", """We study how a regulator can best target inspections. Our case study is a US Occupational Safety and Health Administration (OSHA) program that randomly allocated some inspections. On average, each inspection led to 2.4 (9 percent) fewer serious injuries over the next 5 years. Using new machine learning methods, we find that OSHA could have averted as much as twice as many injuries by targeting inspections to workplaces with the highest expected averted injuries and nearly as many by targeting the highest expected level of injuries. Either approach would have generated up to $850 million in social value over the decade we examine.""", """Increasingly, algorithms are supplanting human decision-makers in pricing goods and services. To analyze the possible consequences, we study experimentally the behavior of algorithms powered by Artificial Intelligence (Q- learning) in a workhorse oligopoly model of repeated price competition. We find that the algorithms consistently learn to charge supracompetitive prices, without communicating with one another. The high prices are sustained by collusive strategies with a finite phase of punishment followed by a gradual return to cooperation. This finding is robust to asymmetries in cost or demand, changes in the number of players, and various forms of uncertainty.""", """ In the rapidly evolving milieu of digital entrepreneurship, the integration of artificial intelligence (AI) technologies, exemplified by ChatGPT, has witnessed burgeoning prominence. However, there remains a dearth of understanding regarding the relationships between ChatGPT adoption in entrepreneurship and individuals cognitive career processes of digital entrepreneurship. Purpose of the article: The primary aim of the research is to adopt the Social Cognitive Career Theory and a moderated mediation model to unravel the intricate dynamics that characterize the impact of ChatGPT adoption in entrepreneurship and digital entrepreneurial intentions, underlying a moderated mediation mechanism of digital entrepreneurial self-efficacy and technostress. Methods: Drawing on the sample of 1326 respondents in Vietnam using a stratified sampling approach, first, Cronbachâ€™s alpha and confirmatory factor analysis were used to test the reliability and validity of scales; after that, Harmanâ€™s single-factor and common latent factor were employed to test the common method bias; finally, the PROCESS macro approach was utilized to test the hypothesized model. Findings & value added: Our findings reveal positive impacts of ChatGPT adoption in entrepreneurship on digital entrepreneurial self-efficacy and digital entrepreneurial intentions. Moreover, digital entrepreneurial self-efficacy is found to significantly mediate the impact of ChatGPT adoption in entrepreneurship on digital entrepreneurial intention. Furthermore, technostress emerges as a significant negative moderator, influencing the impact of ChatGPT adoption in entrepreneurship on both digital entrepreneurial self-efficacy and intentions. This study thus contributes to the literature by advancing our understanding of how AI technologies shape entrepreneurial aspirations, offering valuable insights for scholars and practitioners navigating the transformative landscape of digital entrepreneurship."""]

In [17]:
## HOW MANY TOKENS?

import tiktoken

# define token count function
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# apply to papers
papers['no_tokens'] = papers['Abstract'].apply(lambda x: num_tokens_from_string(str(x), "gpt-4o"))

In [23]:
## ESTIMATE COST

Total = papers['no_tokens'].sum()*0.12
print(f"Number of paper tokens: {Total}")
examples = ''.join([globals()[f'example{i}'] for i in range(1, 15)])
outputs = ''.join([globals()[f'output{i}'] for i in range(1, 15)])
prompt_tokens = num_tokens_from_string(prompt, "gpt-4o") + num_tokens_from_string(examples, "gpt-4o") + num_tokens_from_string(examples, "gpt-4o")
prompt_tokens = prompt_tokens/5
print(f"Number of prompt tokens: {prompt_tokens}")
est_output_tokens = 250
cost = (2.5*prompt_tokens*6000/10**6) + (2.5*Total/10**6) + (10*est_output_tokens*6000/10**6)
print(f"Estimated total cost in $: {cost}")

Number of paper tokens: 1154611.32
Number of prompt tokens: 1085.6
Estimated total cost in $: 34.1705283


In [46]:
## PROMPT GPT-4o MINI

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_papers = pd.read_csv("classed_papers.csv")
    processed_DOIs = set(classed_papers['DOI'])
    data_dict = classed_papers.to_dict('list')
except FileNotFoundError:
    classed_papers = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(papers.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('oops!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_papers = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_papers) - len(pd.read_csv("classed_papers.csv")) == 10:
                    new_papers = classed_papers.tail(10)
                    new_papers.to_csv("classed_papers.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_papers.to_csv("classed_papers.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_papers = pd.DataFrame(data_dict)
classed_papers.to_csv("classed_papers.csv", index=False)
classed_papers

Processing abstracts: 37856it [00:01, 28287.23it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Saved progress at iteration 3

oops!
oops!
oops!
oops!
oops!
oops!
oops!
oops!


INFO:root:Saved progress at iteration 38280


oops!
oops!
oops!
oops!
oops!
oops!
oops!
oops!
oops!
oops!


INFO:root:Saved progress at iteration 38290


...

oops!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39225it [20:16,  1.10it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39226it [20:17,  1.06it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39227it [20:18,  1.05s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39228it [20:19,  1.06s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39229it [20:21,  1.08s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Saved progress at iteration 39230
Processing abstracts: 39230it [20:22,  1.25s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39231it [

oops!
oops!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39348it [22:44,  1.37it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39349it [22:45,  1.25it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Saved progress at iteration 39350
Processing abstracts: 39350it [22:47,  1.10s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39351it [22:48,  1.09s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39352it [22:49,  1.12s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39353it [22:50,  1.10s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 39354it [

,Abstract,DOI,GPT_response,GPT_class,Input_tokens,Output_tokens,reason_stop,system_fingerprint
0,The backdrop of this research is the high glob...,10.1016/j.gfj.2024.101013,This paper investigates the concept of safe-ha...,A,4075.0,66.0,stop,fp_f33667828e
1,The COVID-19 pandemic has highlighted the pern...,10.1057/s41599-024-03083-5,This paper focuses on detecting fake news usin...,B,4143.0,53.0,stop,fp_9722793223
2,This paper explores the ground-breaking approa...,10.1007/s12076-024-00374-y,This paper focuses on the integration of quant...,C,4111.0,62.0,stop,fp_f33667828e
3,Forest restoration has become a worldwide phen...,10.1016/j.forpol.2024.103297,This paper examines the influence of superstit...,B,4058.0,52.0,stop,fp_f33667828e
4,The complexity of emotions has thus far limite...,10.1057/s41599-024-02869-x,This paper proposes a novel theory of emotions...,B,4056.0,65.0,stop,fp_f905cf32a9
...,...,...,...,...,...,...,...,...
39755,The demand prediction becoming an essential to...,10.3846/bm.2020.604,This paper focuses on demand prediction models...,B,4102,66,stop,fp_483d39d857
39756,Systems Thinking theorist J. P. Monat has hypo...,10.1016/j.futures.2024.103429,This paper explores the concept of self-awaren...,B,4114,42,stop,fp_483d39d857
39757,How much can we learn finance directly from te...,10.1287/mnsc.2022.01468,This paper develops a new machine learning fra...,B,4156,69,stop,fp_483d39d857
39758,This paper explores the factors that affect em...,10.1080/02642060801935739,This paper focuses on internal management fact...,B,4099,65,stop,fp_483d39d857


In [5]:
classed_papers = pd.read_csv('classed_papers.csv')
classed_papers

,Abstract,DOI,GPT_response,GPT_class,Input_tokens,Output_tokens,reason_stop,system_fingerprint
0,The backdrop of this research is the high glob...,10.1016/j.gfj.2024.101013,This paper investigates the concept of safe-ha...,A,4075.0,66.0,stop,fp_f33667828e
1,The COVID-19 pandemic has highlighted the pern...,10.1057/s41599-024-03083-5,This paper focuses on detecting fake news usin...,B,4143.0,53.0,stop,fp_9722793223
2,This paper explores the ground-breaking approa...,10.1007/s12076-024-00374-y,This paper focuses on the integration of quant...,C,4111.0,62.0,stop,fp_f33667828e
3,Forest restoration has become a worldwide phen...,10.1016/j.forpol.2024.103297,This paper examines the influence of superstit...,B,4058.0,52.0,stop,fp_f33667828e
4,The complexity of emotions has thus far limite...,10.1057/s41599-024-02869-x,This paper proposes a novel theory of emotions...,B,4056.0,65.0,stop,fp_f905cf32a9
...,...,...,...,...,...,...,...,...
39755,The demand prediction becoming an essential to...,10.3846/bm.2020.604,This paper focuses on demand prediction models...,B,4102.0,66.0,stop,fp_483d39d857
39756,Systems Thinking theorist J. P. Monat has hypo...,10.1016/j.futures.2024.103429,This paper explores the concept of self-awaren...,B,4114.0,42.0,stop,fp_483d39d857
39757,How much can we learn finance directly from te...,10.1287/mnsc.2022.01468,This paper develops a new machine learning fra...,B,4156.0,69.0,stop,fp_483d39d857
39758,This paper explores the factors that affect em...,10.1080/02642060801935739,This paper focuses on internal management fact...,B,4099.0,65.0,stop,fp_483d39d857


In [52]:
# Get Cs
C = classed_papers[classed_papers['GPT_class'] == 'C']
C.to_csv('Cs.csv')

In [13]:
# Get random sample of papers to test

As = classed_papers[classed_papers['GPT_class'] == 'A'].reset_index() # class A
As_to_test = As.sample(n=200, random_state = 123)
As_for_Lucy = As_to_test[0:100]
As_for_John = As_to_test[100:201]

Bs = classed_papers[classed_papers['GPT_class'] == 'B'] # class B
Bs_to_test = Bs.sample(n=200, random_state = 123)
Bs_for_Lucy = Bs_to_test[0:100]
Bs_for_John = Bs_to_test[100:201]

As_for_Lucy.to_csv('As_for_Lucy.csv')
Bs_for_Lucy.to_csv('Bs_for_Lucy.csv')

As_for_John.to_csv('As_for_John.csv')
Bs_for_John.to_csv('Bs_for_John.csv')

In [375]:
test = classed_papers.sample(n=25, random_state = 123)
test.to_csv('for_John.csv')
test1 = classed_papers.sample(n=25, random_state = 123)
test1.to_csv('for_Lucy.csv')

In [24]:
## ROUND 2 WITH GPT 4o: examples

example1 = """Building on administrative data and machine-learning models, I develop a firm-specific measure of regulatory intensity: cost of compliance with all federal paperwork regulations. Regulatory intensity increases the cost of goods sold and overhead spending (SGA). It also incentivizes companies to reduce capital investment, hire fewer employees, and lobby more. The effects are particularly strong among financially constrained firms and those with irreversible investment opportunities, suggesting that regulation affects companies through budgetary pressures and heightened uncertainty. The findings highlight the real effects of regulation and the underlying mechanisms."""

output1 = """The paper uses a machine learning model to develop a firm-specific measure of regulatory intensity. It does not study the economic, financial or management impact or implications of AI, its adoption, or its measurement or policy implications. Class: B"""

example2 = """We investigate the potential implications of large language models (LLMs), such as Generative Pre-trained Transformers (GPTs), on the U.S. labor market, focusing on the increased capabilities arising from LLM-powered software compared to LLMs on their own. Using a new rubric, we assess occupations based on their alignment with LLM capabilities, integrating both human expertise and GPT-4 classifications. … This finding implies that LLM-powered software will have a substantial effect on scaling the economic impacts of the underlying models. We conclude that LLMs such as GPTs exhibit traits of general-purpose technologies, indicating that they could have considerable economic, social, and policy implications."""

output2 = """The paper focuses on predicting the labour market impacts of AI, an economic outcome. Class: A"""

example3 = """We model Artificial Intelligence (AI) as self-learning capital: Its productivity rises by its use and by training with data. In a three-sector model, an AI sector and an applied research (AR) sector produce intermediates for a final good firm and compete for high-skilled workers. AR development benefits from inter-temporal spillovers and knowledge spillovers of agents working in AI, and AI benefits from application gains through its use in AR. … We show that suitable tax policies induce socially optimal movements of workers between sectors. In particular, we provide a macroeconomic rationale for an AI-tax on AI-producing firms, once the accumulation of AI has sufficiently progressed."""

output3 = """This paper develops a theoretical model of AI as capital and discusses tax policy implications. This model is relevant to the labour market and growth, and discusses economic policy implications, either of which qualify it to be in A. Class: A."""

example4 = """The intermittent and uncontrollable power output from the ever-increasing renewable energy sources, require large amounts of operating reserves to retain the system frequency within its nominal range. … In this work, we aim at delivering real-time load forecasting to lower the operating-reserve requirements based on intra-hour weather update predictors. Based on critical predictors and their historical data, we train an artificial model that is able to forecast the load ahead with great accuracy. This is a feed-forward neural network with two hidden layers, which performs real-time forecasts with the aid of a predictive model control developed to update the recommendations intra-hourly …. Performing daily simulations for an annual time-horizon, we observe that significant improvements exist in terms of decreased operating reserve requirements to regulate the violated frequency. …"""

output4 = """This paper trains a neural network to forecast energy load. It does not directly investigate the economic, financial or management impacts of AI; measurement challenges, or its adoption. Class: B"""

example5 = """This article surveyed different emerging technologies (ET), in particular artificial intelligence, and their burgeoning application in the evaluation industry. Evidence suggests that evaluators have been relatively slow in adopting ET in their practice. However, more recent data suggest that ET adoption is increasing. This article then analyzed if, and how, ET affect the evaluation industry and evaluation practice. The article finds that program evaluation is one of several competing forms of knowledge production informing decision-making, particularly in the government and not-for-profit sectors. … In this article, it is argued that evaluators must, albeit critically, embrace ET. Most likely, ET will complement evaluation practice and, in some instances, displace human tasks."""

output5 = """This paper analyses the application and adoption of emerging technologies, especially AI, to the evaluation industry. Class: A"""

example6 = """Deep learning, the most important subfield of machine learn- ing and artificial intelligence (AI) over the last decade, is considered one of the fundamental technologies underpinning the Fourth Industrial Revolution. But despite its record-breaking history, deep learning’s enormous appetite for compute and data means that sometimes it can be too costly to practically use. In this paper, we connect technical insights from deep learning scaling laws and transfer learning with the economics of IT to propose a framework for estimating the cost of deep learning computer vision systems that achieve a desired level of accuracy. Our tool can be of practical use to AI practitioners in industry or academia to guide investment decisions."""

output6 = """This paper is about one of the drivers of progress in deep learning (i.e., data and compute). This is about the economic drivers of AI progress. Class: A"""

example7 = """Innovations in statistical technology, including in predicting creditworthiness, have sparked concerns about distributional impacts across categories such as race. Theoretically, distributional consequences of better statistical technology can come from greater flexibility to uncover structural relationships, or from triangulation of otherwise excluded characteristics. Using data on US mortgages, we predict default using traditional and machine learning models. We find that Black and Hispanic borrowers are disproportionately less likely to gain from the introduction of machine learning. In a simple equilibrium credit market model, machine learning increases disparity in rates between and within groups; these changes are primarily attributable to greater flexibility."""

output7 = """The paper analyses the distributional impacts of the adoption of AI for the prediction of mortgage defaults. It focuses on economically-relevant ethical implications of AI, which qualifies it to be in class A. Class: A."""

example8 = """This manuscript presents a methodical examination of the utilization of Artificial Intelligence (AI) in the assessment of emotions in texts related to healthcare, with a particular focus on the incorporation of Natural Language Processing (NLP) and deep learning technologies. We scrutinize numerous research studies that employ AI to augment sentiment analysis, categorize emotions, and forecast patient outcomes based on textual information derived from clinical narratives, patient feed- back on medications, and online health discussions. The review demonstrates noteworthy progress in the precision of algorithms used for sentiment classification, the prognostic capabilities of AI models for neurodegenerative diseases, and the creation of AI- powered systems that offer support in clinical decision-making. …."""

output8 = """This paper examines the use of AI to analyse emotions in healthcare. It does not directly study the economic, financial or management impact or implications of AI, its adoption, or its measurement or policy implications. Class: B"""


In [50]:
prompt = "You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. \n\nCategory A: The paper studies: \n- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. \n- The measurement of the economic impact of AI, including measurement challenges. \n- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship between AI, racial inequality and income inequality. \n- The adoption of AI by firms, the government, or sectors; what affects adoption decisions; and how AI is used. \n- Economic drivers of AI progress, such as cost of compute. \nProvided they meet the earlier requirements, include both theoretical and empirical, micro and macro studies from the fields of economics, management, finance, and innovation (including sector-specific studies). \n\nCategory B: The paper does not focus on the above topics. Category B includes papers that focus on developing machine learning techniques to predict financial returns. Category B also includes papers that focus on social outcomes such as race, religion and politics where the paper does not make explicit links to economic outcomes, such as economic inequality, unemployment or resource distribution. Also include papers that are not about the impact or implications of AI, e.g., those that focus on developing or using AI econometric techniques other than to analyse the impact of implications of AI. \n\nCategory C: It is difficult to classify the paper between A and B based on the abstract. \n\nInstructions: 1. Reason first, and summarise in no more than three bullet points, the main focus and methodology of the paper outlined in the abstract. 2. Only when you have enough information, decide on a classification based on 1. Do not use more than 100 words in your response. When giving your final classification, format it as 'Class: x' where x is your suggested classification. For each abstract you correctly classify, I will pay you $10 as a tip."

In [51]:
print(prompt)

You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. 

Category A: The paper studies: 
- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. 
- The measurement of the economic impact of AI, including measurement challenges. 
- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship bet

In [45]:
## FOR TESTING

test = As.sample(n=50, random_state = 1339)
test = test.reset_index()
print(test)

    index                                           Abstract  \
0   15800  Due to the global warming, a significant reduc...   
1   35480  This study explored the challenges and recomme...   
2   36945  Purpose: The digital revolution has changed co...   
3    7352  In order to improve the ability of enterprises...   
4   27542  Purpose: While customer experience (CE) is rec...   
5     217  We investigate the impacts of robot adoption (...   
6   11908  Adoption of artificial intelligence (AI) is ca...   
7   26261  In light of the emerging discourse on AI syste...   
8   11373  The aim of this study is to elucidate the conc...   
9    7926  Studies of economic growth often refer to “gen...   
10  34031  Healthcare lags decades behind other industrie...   
11  12305  As customer demands for personalised items con...   
12   6910  Geographical Indications (GIs), as Protected D...   
13  24553  Industrial-Internet-of-things (IIOT) architect...   
14  39452  A fundamental problem in the 

In [63]:
prompt = "You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. \n\nCategory A: The paper studies: \n- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. \n- The measurement of the economic impact of AI, including measurement challenges. \n- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship between AI, racial inequality and income inequality. \n- The adoption of AI by firms, the government, or sectors; what affects adoption decisions; and how AI is used. \n- Economic drivers of AI progress, such as cost of compute. \nProvided they meet the earlier requirements, include both theoretical and empirical, micro and macro studies from the fields of economics, management, finance, and innovation (including sector-specific studies). \n\nCategory B: The paper does not fit into category A. This includes papers that develop or apply a machine learning technique, or measure to study an outcome other than the impact of AI, and papers that develop an AI tool. This includes papers that focus on developing machine learning techniques to predict financial returns. It also includes papers that focus on social outcomes such as race, religion and politics where the paper does not make explicit links to economic outcomes, such as economic inequality, unemployment or resource distribution. Include in this category papers that are not about AI. \n\nInstructions: 1. Reason first, and summarise in no more than three bullet points, the main focus and methodology of the paper outlined in the abstract. 2. Only when you have enough information, decide on a classification based on 1. Do not use more than 100 words in your response. When giving your final classification, format it as 'Class: x' where x is your suggested classification. For each abstract you correctly classify, I will pay you $10 as a tip."


In [69]:
print(prompt)

You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. 

Category A: The paper studies: 
- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. 
- The measurement of the economic impact of AI, including measurement challenges. 
- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship bet

In [70]:
example1 = """We investigate the potential implications of large language models (LLMs), such as Generative Pre-trained Transformers (GPTs), on the U.S. labor market, focusing on the increased capabilities arising from LLM-powered software compared to LLMs on their own. Using a new rubric, we assess occupations based on their alignment with LLM capabilities, integrating both human expertise and GPT-4 classifications. Our findings reveal that around 80% of the U.S. workforce could have at least 10% of their work tasks affected by the introduction of LLMs, while approximately 19% of workers may see at least 50% of their tasks impacted. We do not make predictions about the development or adoption timeline of such LLMs. The projected effects span all wage levels, with higher-income jobs potentially facing greater exposure to LLM capabilities and LLM-powered software. Significantly, these impacts are not restricted to industries with higher recent productivity growth. Our analysis suggests that, with access to an LLM, about 15% of all worker tasks in the US could be completed significantly faster at the same level of quality. When incorporating software and tooling built on top of LLMs, this share increases to between 47 and 56% of all tasks. This finding implies that LLM-powered software will have a substantial effect on scaling the economic impacts of the underlying models. We conclude that LLMs such as GPTs exhibit traits of general-purpose technologies, indicating that they could have considerable economic, social, and policy implications."""

output1 = """The paper focuses on predicting the labour market impacts of AI, an economic outcome. It uses GPT-4 classifications to estimate occupational AI exposure. Class: A"""

example2 = """The use of Markov processes (or Markov chains) has become widespread in dynamic stochastic modeling. For example, its use is ubiquitous in macroeconomics (dynamic stochastic general equilibrium), finance (dynamic asset pricing), and areas of microeconomics (dynamic programming). As we discuss below, its application in dynamic land use has been more limited, but is, in principle, no less applicable. Using a multi-nominal logit (ML) specification together with serial data on agricultural land use from California, we estimate Markov transition probabilities conditional on number of exogenous factors. Applying so-called “first step” analysis, these transition probabilities are used to forecast the distribution of agricultural crops, which in turn can be used for policy making."""

output2 = """This paper focuses on developing a dynamic stochastic modelling method. It applies this to modelling land use. While land use may have productivity implications, the paper is not focused on analysing the impact of AI on this or any other relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B."""

example3 = """We model Artificial Intelligence (AI) as self-learning capital: Its productivity rises by its use and by training with data. In a three-sector model, an AI sector and an applied research (AR) sector produce intermediates for a final good firm and compete for high-skilled workers. AR development benefits from inter-temporal spillovers and knowledge spillovers of agents working in AI, and AI benefits from application gains through its use in AR. … We show that suitable tax policies induce socially optimal movements of workers between sectors. In particular, we provide a macroeconomic rationale for an AI-tax on AI-producing firms, once the accumulation of AI has sufficiently progressed."""

output3 = """This paper develops a theoretical model of AI as capital and discusses tax policy implications. This model is relevant to the labour market and growth, and discusses economic policy implications, either of which qualify it to be in A. Class: A."""

example4 = """The COVID-19 crisis has had a tremendous economic impact for all countries. Yet, assessing the full impact of the crisis has been frequently hampered by the delayed publication of official GDP statistics in several emerging market and developing economies. This paper outlines a machine-learning framework that helps track economic activity in real time for these economies. As illustrative examples, the framework is applied to selected sub-Saharan African economies. The framework is able to provide timely information on economic activity more swiftly than official statistics."""

output4 = """This paper proposes a method to predict the economic impact of Covid-19. It uses a machine-learning framework to generate real-time forecasts. It is not focused on analysing the impact of AI on any economically relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B"""

example5 = """Innovations in statistical technology, including in predicting creditworthiness, have sparked concerns about distributional impacts across categories such as race. Theoretically, distributional consequences of better statistical technology can come from greater flexibility to uncover structural relationships, or from triangulation of otherwise excluded characteristics. Using data on US mortgages, we predict default using traditional and machine learning models. We find that Black and Hispanic borrowers are disproportionately less likely to gain from the introduction of machine learning. In a simple equilibrium credit market model, machine learning increases disparity in rates between and within groups; these changes are primarily attributable to greater flexibility."""

output5 = """The paper analyses the distributional impacts of the adoption of AI for the prediction of mortgage defaults. It does so using an ML model. It focuses on economically-relevant ethical implications of AI, which qualifies it to be in class A. Class: A."""

example6 = """This paper introduces a novel geo-connected hierarchical deep learning framework for enhanced flood risk modeling, leveraging high-resolution meteorological and hydraulic data. Our method adopts a geographically structured hierarchical deep learning model, providing a more refined analysis of flood risks. In a case study using the Mississippi River, we demonstrate superior performance over traditional deep learning models in terms of accuracy and predictive power. Significantly, when applied to the National Flood Insurance Program’s policy and claim data, our model not only yields more precise risk pricing but also effectively reduces the solvency capital requirement by aligning net premiums more closely with actual risks. The framework’s global optimization approach effectively harnesses spatial and temporal data, while its physics-based structure offers improved interpretability and addresses the complex challenges of anthropogenic influences in flood risk assessment."""

output6 = """This paper focuses on modelling flood risk. It does so using a deep learning model. It is not focused on analysing the impact of AI on any economically relevant outcome, or the implications of AI for policy, measurement, or ethics. Class: B"""

example7 = """Leveraging the blood-making function of public transfer payments and stimulating the endogenous motivation of beneficiaries are key focus for poverty alleviation worldwide. Based on the data from 2014‚2018 China Family Panel Studies (CFPS) and machine learning algorithms, this article comprehensively evaluate the effect of Chinese public transfer payment system on labour supply. We find that public transfer payments oriented households increase the labour supply of family members. Decomposing the comprehensive effect, public transfer payments increase the employment probability at the extensive margin and extend the working hours at the intensive margin. Mechanism analysis indicates that public transfer payments is conducive to enhance health capital, promote labour skill training, and relax constraints on infant and child care, thus improving labour supply. Furthermore, groups in female, young, and education below high school show a higher incentive effect for labour supply."""

output7 = """This paper focuses on making public transfer payments more effective at poverty alleviation. It uses a machine learning technique to evaluate the effect of Chinese public transfer payments on labour supply. While the topic is economically relevant, it is not about the impact of AI - it just uses a machine learning technique. Class: B"""

example8 = """Artificial intelligence (AI) and machine learning (ML) algorithms have transformed various industries, including healthcare. Healthcare organizations are now using AI and ML algorithms to drive strategic leadership and decision-making, as they provide insights that help organizations manage resources, improve patient outcomes, and increase efficiency. This research paper examines how AI and ML algorithms are used in healthcare to drive strategic leadership. The paper also explores the benefits and challenges associated with using these technologies in healthcare. The study found that AI and ML algorithms can help healthcare organizations make data-driven decisions, optimize resource allocation, and improve patient outcomes. However, there are still challenges related to data quality and privacy that must be addressed to ensure that AI and ML algorithms are used effectively in healthcare."""

output8 = """This paper focuses on the adoption of algorithms in a particular sector, healthcare. It discusses the adoption or diffusion of AI and discusses relevant management outcomes (strategic leadership and decision-making). Class: A"""

example9 = """General purpose technologies (GPTs) such as AI enable and require significant complementary investments, including co-invention of new processes, products, business models and human capital. These complementary investments are often intangible and poorly measured in the national accounts, even when they create valuable assets for the firm. We develop a model that shows how this leads to an underestimation of productivity growth in the early years of a new GPT, and how later, when the benefits of intangible investments are harvested, productivity growth will be overestimated. Our model generates a Productivity J-Curve that can explain the productivity slowdowns often accompanying the advent of GPTs, as well as the increase in productivity later. We use our model to analyze empirically the historical roles of intangibles tied to R&D, software, and computer hardware. We find substantial and ongoing Productivity J-Curve effects for software in particular and computer hardware to a lesser extent. Our adjusted measure TFP is 11.3% higher than official measures at the end of 2004, and 15.9% higher than official measures at the end of 2017. We then assess how AI-related intangible capital may be currently affecting measured productivity and find the effects are small but growing."""

output9 = """This paper is about AI and a productivity J-curve. It develops a model to analyse this. It is about the measurement challenges AI poses as well as the impact of AI on productivity. Class: A"""

example10 = """Deep learning, the most important subfield of machine learn- ing and artificial intelligence (AI) over the last decade, is considered one of the fundamental technologies underpinning the Fourth Industrial Revolution. But despite its record-breaking history, deep learning’s enormous appetite for compute and data means that sometimes it can be too costly to practically use. In this paper, we connect technical insights from deep learning scaling laws and transfer learning with the economics of IT to propose a framework for estimating the cost of deep learning computer vision systems that achieve a desired level of accuracy. Our tool can be of practical use to AI practitioners in industry or academia to guide investment decisions."""

output10 = """This paper is about one of the drivers of progress in deep learning (i.e., data and compute). The paper uses scaling laws to estimate the cost of deep learning computer vision systems. This is about the economic drivers of AI progress. Class: A"""

example11 = """The intermittent and uncontrollable power output from the ever-increasing renewable energy sources, require large amounts of operating reserves to retain the system frequency within its nominal range. … In this work, we aim at delivering real-time load forecasting to lower the operating-reserve requirements based on intra-hour weather update predictors. Based on critical predictors and their historical data, we train an artificial model that is able to forecast the load ahead with great accuracy. This is a feed-forward neural network with two hidden layers, which performs real-time forecasts with the aid of a predictive model control developed to update the recommendations intra-hourly …. Performing daily simulations for an annual time-horizon, we observe that significant improvements exist in terms of decreased operating reserve requirements to regulate the violated frequency. …"""

output11 = """This paper trains a neural network to forecast energy load. It does not directly investigate the economic, financial or management impacts of AI; measurement challenges, or its adoption. Class: B"""

example12 = """High-frequency trading is prevalent, where automated decisions must be made quickly to take advantage of price imbalances and patterns in price action that forecast near-future movements. Although many algorithms have been explored and tested, analytical methods fail to harness the whole nature of the market environment by focusing on a limited domain. With the ever-growing machine learning field, many large-scale end-to-end studies on raw data have been successfully employed to increase the domain scope for profitable trading but are very difficult to replicate. Combining deep learning on the order books with reinforcement learning is one way of breaking down large-scale end-to-end learning into more manageable and lightweight components for reproducibility suitable for retail trading. The following article focuses on forecasting returns across multiple horizons using order flow imbalance and training three temporal-difference learning models for five financial instruments to provide trading signals. Through backtesting and forward testing, the results prove potential but require further minimal modifications for consistently profitable trading to fully handle retail trading costs, slippage, and spread fluctuation.  © 2024 With Intelligence LLC."""

output12 = """This paper focuses on using machine learning, specifically deep learning and reinforcement learning, to improve high-frequency trading strategies. The outcome it explores is forecasted returns across multiple time horizions. The paper focuses on developing a machine learning technique for financial forecasting and does not discuss wider implications for economic outcomes, measurement, AI progress, policy or ethics. Class: B"""

example13 = """The General Data Protection Regulation (GDPR) of the European Union has established regulations on automated decisions in Article 22 with the proliferation of artificial intelligence. In response, the Personal Information Protection Act (PIPA) of South Korea, serving as a counterpart to the GDPR, has recently incorporated provisions for automated decisions under Article 37-2 through an amendment. Although the PIPA follows a distinct legal framework from the GDPR, it is crucial to ensure an equivalent level of protection for fundamental rights. Recognising this concern, this study analyses the differences between the PIPA and GDPR regarding automated decisions, focusing on three aspects: format, target, and content. This analysis identifies that the PIPA lacks comprehensive safeguards for data subjects in certain aspects compared to the GDPR. First, regarding the format, the PIPA grants the right to object rather than establishing a general prohibition to automated decisions, posing limitations in protecting individuals who are unable to effectively exercise their rights. Second, in terms of the target, the PIPA regulates a completely automated status at the overall system level, creating a regulatory vacuum for a multi-stage profiling system. Third, concerning the content, the PIPA faces several technical and practical limitations that remain unresolved in delineating the content of the right to explanation. Building upon this analysis, this study proposes potential legislation and interpretation remedies to address these concerns based on each aspect."""

output13 = """This paper analyses the differences between the EU General Data Protection Regulation (GDPR) and South Korea's Personal Information Protection Act (PIPA) with regards to automated decision-making from AI. While it discusses the policy implications of AI, the paper does not focus on the links between these policies and economic outcomes. Class: B"""

example14 = """Building on administrative data and machine-learning models, I develop a firm-specific measure of regulatory intensity: cost of compliance with all federal paperwork regulations. Regulatory intensity increases the cost of goods sold and overhead spending (SGA). It also incentivizes companies to reduce capital investment, hire fewer employees, and lobby more. The effects are particularly strong among financially constrained firms and those with irreversible investment opportunities, suggesting that regulation affects companies through budgetary pressures and heightened uncertainty. The findings highlight the real effects of regulation and the underlying mechanisms."""

output14 = """The paper uses a machine learning model to develop a firm-specific measure of regulatory intensity. It does not study the economic, financial or management impact or implications of AI, its adoption, or its measurement or policy implications. Class: B"""

In [74]:
As = As.reset_index()

In [77]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Processing abstracts: 2720it [00:00, 14727.28it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 5271it [00:01, 3047.08it/s] INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.

,Abstract,DOI,GPT_response,GPT_class,Input_tokens,Output_tokens,reason_stop,system_fingerprint
0,The backdrop of this research is the high glob...,10.1016/j.gfj.2024.101013,This paper investigates the properties of East...,B,3921,63,stop,fp_71fab63729
1,Artificial Intelligence (AI)-driven language m...,10.1057/s41599-024-02720-3,This paper assesses the global consequences of...,A,3948,50,stop,fp_71fab63729
2,This paper aims to estimate the returns to sol...,10.1016/j.eneco.2024.107768,This paper estimates the economic impact of so...,A,3889,60,stop,fp_71fab63729
3,Fostering innovation is one of the key roles o...,10.1016/j.jeca.2024.e00372,This paper investigates the impact of investme...,A,3938,68,stop,fp_71fab63729
4,Employment is a pivotal driver for ensuring an...,10.1057/s41599-024-03333-6,This paper examines the impact of robot applic...,A,3955,59,stop,fp_71fab63729
...,...,...,...,...,...,...,...,...
6286,This paper analyses the impact of intangibles ...,10.1080/13662716.2022.2138280,This paper examines the impact of intangible c...,B,3881,62,stop,fp_71fab63729
6287,While algorithmic management provides new opti...,10.1016/j.tourman.2022.104716,This paper explores the impact of algorithmic ...,A,3890,69,stop,fp_71fab63729
6288,In order to investigate the acceptance of a Pa...,10.1016/j.tra.2022.01.009,This paper investigates the acceptance of a Pa...,A,3968,63,stop,fp_71fab63729
6289,Learning consumers' personalized privacy prefe...,10.1287/isre.2023.0318,This paper proposes a framework for predicting...,B,3981,66,stop,fp_71fab63729


In [76]:
backup_As = classed_As.copy()

In [79]:
test = classed_As.sample(n=50, random_state = 100)
test.to_csv('test_round2.csv')

In [80]:
len(classed_As[classed_As['GPT_class'] == 'A'])/len(classed_As)

0.7924018439039898

In [82]:
len(classed_As[classed_As['GPT_class'] == 'A'])

4985

In [83]:
classed_As[classed_As['GPT_class'] == 'A'].to_csv('As_final_sample.csv')